In [1]:
import numpy as np 

In [2]:
class Robot_Kinematics:
    
    def __init__(self,k_arm,t_arm):
        self.k = np.array(k_arm)
        self.t = np.array(t_arm)
        self.N_joints = k_arm.shape[0]
 
    def rot(self,k,q):
        '''
        INPUT:
        k - A 3 element array having the unit axis to rotate around (kx, ky, kz)
        q - Rotation angle
        
        OUTPUT:
        Returns a 3x3 rotation matrix
        '''
        
        c_theta = np.cos(q)
        s_theta = np.sin(q)
        v_theta = 1 - np.cos(q)
        kx = k[0]
        ky = k[1]
        kz = k[2]        
        r00 = kx * kx * v_theta + c_theta
        r01 = kx * ky * v_theta - kz * s_theta
        r02 = kx * kz * v_theta + ky * s_theta
        r10 = kx * ky * v_theta + kz * s_theta
        r11 = ky * ky * v_theta + c_theta
        r12 = ky * kz * v_theta - kx * s_theta
        r20 = kx * kz * v_theta - ky * s_theta
        r21 = ky * kz * v_theta + kx * s_theta
        r22 = kz * kz * v_theta + c_theta
        Rot_Matrix = np.array([[r00, r01, r02],[r10, r11, r12],[r20, r21, r22]])                                                     
        return Rot_Matrix
    
    def HT_Matrix(self,k,t,q):
        '''
        INPUT:
        k - A 3 element array having the unit axis to rotate around (kx, ky, kz)
        q - Rotation angle
        t - Translation from current frame to next frame
        
        OUTPUT:
        Returns a 4x4 homogenous transformation matrix
        '''
        R = self.rot(k,q) # Rotation matrix
        T = np.array([[t[0]],[t[1]],[t[2]]]) # Translation part
        homgen_mat = np.concatenate((R, T), axis=1) 
        extra_row_homgen = np.array([[0, 0, 0, 1]])  
        homgen_mat = np.concatenate((homgen_mat, extra_row_homgen), axis=0)
        return homgen_mat
    
    def pos(self,angle,idx=-1,p_i=[0,0,0]):
        '''
        Computes the position in global frame of a point in joint frame
        
        INPUT:
        p_i - A 3 element vector having the position in the frame of the index joint
        idx - index of the joint frame that is converting from joint 0 to N_joints -1
        
        OUTPUT:
        Returns a 3 element vector having position coordinates with respect to base frame
        
        '''
        # Position of the current joint - as represented by the idx
        this_joint_position = np.array([[p_i[0]],[p_i[1]],[p_i[2]],[1]])  
        
        # End-effector joint (-1 index indicates that it is the last joint)
        if (idx == -1):
            idx = self.N_joints - 1   
            
        # Original idex of the current joint
        orig_joint_index = idx  
        
        temp = None 
        # Starting from the index of the current joint and go backwards to index 0
        while (idx >= 0):           
            # If the current index is same as original joint index
            if (idx == orig_joint_index):
                temp = self.HT_Matrix(self.k[idx],self.t[idx],angle[idx]) @ this_joint_position
             # If the current index is not same as original joint index
            else: 
                temp = self.HT_Matrix(self.k[idx],self.t[idx],angle[idx]) @ temp          
            idx = idx - 1
        # Getting the positions with respect to the global frame
        position_global_frame = np.array([temp[0][0], temp[1][0], temp[2][0]])        
        
        return position_global_frame
 
    def pseudo_inv(self,theta_initial,p_eff_N,target_pos,max_steps=np.inf):
        '''
        Conputes inverse kinematics of the jacobian matrix
        
        INPUT:
        theta_initial - An N-element array having current joint angles in radians
        p_eff_N - A 3 element vector having translation from the last joint to the end effector in the last joints frame of reference
        max_steps - Maximum number of iterations to compute
        
        OUTPUT:
        Returns an N-element array having the joint angles that result in the end-effector reaching target position
        '''
        v_step_size = 0.05
        theta_max_step = 0.2
        p_j = self.pos(theta_initial,p_i=p_eff_N)   
        delta_p = target_pos - p_j # Distance between current position of end-effector and target position
        j = 0 
        while np.linalg.norm(delta_p) > 0.01 and j<max_steps:         
            v_p = delta_p * v_step_size / np.linalg.norm(delta_p)         
            J_j = self.jacobian(theta_initial,p_eff_N)
            J_invj = np.linalg.pinv(J_j)
            v_Q = np.matmul(J_invj,v_p)
            theta_initial = theta_initial + np.clip(v_Q,-1*theta_max_step,theta_max_step)
            p_j = self.pos(theta_initial,p_i=p_eff_N)          
            j = j + 1
            delta_p = target_pos - p_j 
        return theta_initial
 
    def jacobian(self,angle,p_eff_N=[0,0,0]):
        '''
        INPUT:
        angle - An N element array having the current joint angles in radians
        p_eff_N - A 3 element vector having translation from the last joint to the end effector in the last joints frame of reference
        
        OUTPUT:
        Returns a 3xN 2D matrix which is the Jacobian matrix
        '''
        p_eff = self.pos(angle,-1,p_eff_N) # End-effector position in global frame    
        first_iter = True         
        J = None         
        for i in range(0, self.N_joints):
            if (first_iter == True):
                # Diff between end-effector position in global frame and the current joint in global frame
                p_eff_minus_this_p = p_eff - self.pos(angle,idx=i)
 
                kx = self.k[i][0]
                ky = self.k[i][1]
                kz = self.k[i][2]
                k = np.array([kx, ky, kz])

                px = p_eff_minus_this_p[0]
                py = p_eff_minus_this_p[1]
                pz = p_eff_minus_this_p[2]
                p_eff_minus_this_p = np.array([px, py, pz])

                this_jacobian = np.cross(k, p_eff_minus_this_p) 
                this_jacobian = np.array([[this_jacobian[0]],[this_jacobian[1]],[this_jacobian[2]]])                                                                                             
                J = this_jacobian
                first_iter = False
            else:
                p_eff_minus_this_p = p_eff - self.pos(angle,idx=i)

                kx = self.k[i][0]
                ky = self.k[i][1]
                kz = self.k[i][2]
                k = np.array([kx, ky, kz])
                
                # Diff between the current joint's position and end-effector position
                px = p_eff_minus_this_p[0]
                py = p_eff_minus_this_p[1]
                pz = p_eff_minus_this_p[2]
                p_eff_minus_this_p = np.array([px, py, pz])

                this_jacobian = np.cross(k, p_eff_minus_this_p) 
                this_jacobian = np.array([[this_jacobian[0]],[this_jacobian[1]],[this_jacobian[2]]])                                                                                           
                J = np.concatenate((J, this_jacobian), axis=1)           
        return J
    
    def Inverse_Kinematics(self,goal_position):
        a1 = 4.7
        a2 = 5.9
        a3 = 5.4
        a4 = 6.0
        # Position of end effector in joint 2 (i.e. the last joint) frame
        p_eff_2 = [a4,0,a3]
 
        # Starting joint angles in radians (joint 1, joint 2)
        q_0 = np.array([0,0,0,0,0,0])
            
        # Return joint angles that result in the end effector reaching endeffector_goal_position
        final_q = self.pseudo_inv(q_0, p_eff_N = p_eff_2, target_pos = goal_position, max_steps=500)
        final_q = np.degrees(final_q)
        
        # Display the starting position of each joint in the global frame
        for i in np.arange(0,self.N_joints):
            print('Joint',i,'position:',self.pos(q_0,idx=i))
        
        print('\nEnd-effector position:',self.pos(q_0,idx=-1,p_i=p_eff_2))

        # Final Joint Angles in degrees   
        print('\nFinal Joint Angles in Degrees')
        for u in range(0,self.N_joints):
            print('Theta',u,':',final_q[u])

In [3]:
def main():
    # k - array that lists different axes of rotation  
    k = np.array([[0,0,1],[0,0,1],[0,0,1],[0,0,1],[0,0,1],[0,0,1]])
    a1 = 4.7
    a2 = 5.9
    a3 = 5.4
    a4 = 6.0
    # t - array that lists translations from previous joint to current joint
    t = np.array([[0,0,0],[a2,0,a1],[a3,a4,0],[a1,a4,0],[0,a4,0],[a2,0,a3]])
    #goal_position = np.array([4.0,10.0,a1 + a4])
    goal_position = np.array([0.35, 0.1, 0.5])
    
    # Creating an object and calling the method
    obj = Robot_Kinematics(k,t)
    print('INVERSE KINEMATICS FOR GIVEN TARGET POSITION\n')
    print('\nTarget position:',goal_position,'\n')
    obj.Inverse_Kinematics(goal_position)    

In [4]:
if __name__ == '__main__':
    main()

INVERSE KINEMATICS FOR GIVEN TARGET POSITION


Target position: [0.35 0.1  0.5 ] 

Joint 0 position: [0. 0. 0.]
Joint 1 position: [5.9 0.  4.7]
Joint 2 position: [11.3  6.   4.7]
Joint 3 position: [16.  12.   4.7]
Joint 4 position: [16.  18.   4.7]
Joint 5 position: [21.9 18.  10.1]

End-effector position: [27.9 18.  15.5]

Final Joint Angles in Degrees
Theta 0 : -17.656066838466167
Theta 1 : 26.205193790300097
Theta 2 : -5.288507106973826
Theta 3 : -37.27305087138466
Theta 4 : -76.82592882278529
Theta 5 : -31.600578665420596
